# Task 1

In [1]:
%matplotlib nbagg

In [2]:
import numpy as np
import strid
import matplotlib.pyplot as plt
import scipy.signal

/Users/emilneby/miniforge3/envs/tkt4550/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Generate random vibration data for a shear frame of 8 storeys

Based on example notebook "00-generating-data.ipynb"

In [3]:
# Create a shear frame
sf = strid.utils.ShearFrame(8, 1e3, 1e7) #Defining 8 storeys here
sf.set_rayleigh_damping_matrix([sf.get_natural_frequency(1), sf.get_natural_frequency(sf.n)], [.05]*2)

# Determine the time discretization and period
Tmax = 1. / strid.w2f(sf.get_natural_frequency(1))
fmax = strid.w2f(sf.get_natural_frequency(sf.n))
T = 1000*Tmax
fs = 5 * fmax
t = np.arange(0., T, 1/fs)

# Define loads on system
## Unmeasureable: Stochastic loads on all floors (Process noise)
w = np.random.normal(size=(sf.n, t.size)) * 1e-1

## Load matrix, f
F = w.copy()

# Simulate response, accelerations at each floor measured
y0, _, _ = sf.simulate(t, F)

noise_std = y0.std()

# Add measurement noise
v = np.random.normal(size=y0.shape)*noise_std
y = y0 + v

plt.figure("Accelerations measured at top floor")
plt.plot(t, y[-1], label="w/noise")
plt.plot(t, y0[-1], label="wo/noise")
plt.legend()
plt.grid()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.figure("PSD of accelerations at top floor")
for yi in [y[-1], y0[-1]]:
    freqs, Gyy = scipy.signal.welch(yi, fs, nperseg=2**10)
    plt.semilogy(freqs, Gyy)


for n in range(1, 1+sf.n):
    plt.axvline(strid.w2f(sf.get_natural_frequency(n)), alpha=.3)
plt.ylabel('PSD')
plt.xlabel('Frequency (Hz)')
plt.grid()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
true_frequencies = np.array([sf.get_natural_frequency(i)/(2*np.pi) for i in range(1, sf.n+1)])
true_damping = np.array([sf.get_rayleigh_damping_ratio(i) for i in range(1, sf.n+1)])
true_modeshapes = np.array([sf.get_mode_shape(i) for i in range(1, sf.n+1)])

#Saving the data
np.savez("results/data-stochastic_8_floor.npz",
         y=y, fs=fs,
         true_frequencies=true_frequencies,
         true_damping=true_damping,
         true_modeshapes=true_modeshapes
        )

### Use covariance-driven stochastic subspace identification to perform system identification

Based on example notebook "01a-stochastic-system-identification-CovSSI.ipynb"

In [5]:
data = np.load("results/data-stochastic_8_floor.npz")
y = data["y"]
fs = data["fs"]
true_f = data["true_frequencies"]
true_xi = data["true_damping"]
true_modeshapes = data["true_modeshapes"]

In [6]:
ssid = strid.CovarianceDrivenStochasticSID(y, fs)

In [7]:
modes = {}
for i, order in enumerate(range(5, 50, 1)):
    A, C, G, R0 = ssid.perform(order, 25)
    modes[order] = strid.Mode.find_modes_from_ss(A, C, ssid.fs)

In [8]:
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(modes)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

# Task 2
### Task 2.1 - finding relative difference

In [9]:
cluster_meat = []
num_modes = 0

for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    num_modes += len(modes_of_order)
    counter = 0
    for mode in modes_of_order:
        counter += 1
        neighbour = strid.find_nearest_neighbour(mode, modes[order-1])

        if (np.isnan(strid.rel_diff_freq(neighbour.f, mode.f))):
            mode.delta_frequency = 1
        else:
            mode.delta_frequency = strid.rel_diff_freq(neighbour.f, mode.f)

        if (np.isnan(np.abs(neighbour.xi - mode.xi))):
            mode.delta_damping = 1
        else:
            mode.delta_damping = np.abs(neighbour.xi - mode.xi)

        if (np.isnan(strid.utils.modal_assurance_criterion(neighbour.v, mode.v))):
            mode.mac = 1
        else:
            mode.delta_mac = strid.utils.modal_assurance_criterion(neighbour.v, mode.v)

        cluster_meat.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])

cluster_meat = np.array(cluster_meat)
# !!! Have to handle modes in the lowest order!!!

/Users/emilneby/PyCharmProjects/TKT4550_Prosjekt/strid/utils.py:728: RuntimeWarning: invalid value encountered in double_scalars
  return self.eigenvalue.imag / np.sqrt(1-self.xi**2)


In [10]:
#print(cluster_meat[:,0])

In [11]:
#Making a 3D scatterplot of frequency, damping and MAC number
fig = plt.figure()
ax = plt.axes(projection='3d')
fg = ax.scatter3D(cluster_meat[:,0], cluster_meat[:,1], cluster_meat[:,2])
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")

print("Total number of modes = " +str(cluster_meat.shape[0]))

<IPython.core.display.Javascript object>

Total number of modes = 1210


### Task 2.2 -  Using K-means clustering to separate all the poles into two groups

In [12]:
# https://scikit-learn.org/stable/modules/clustering.html#k-means
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=2, random_state=0).fit(cluster_meat)
labels = kmeans.labels_

In [13]:
#Assign label to each mode
physical_coordinates = []
mathematical_coordinates = []
count = 0
physical_modes_dict = {}
physical_modes_list = []
num_modes = 0
for order in range(49, 5, -1):
    modes_of_order = modes[order]
    #print("Order: " + str(order))
    physical_modes_in_order = []
    for mode in modes_of_order:
        mode.physical = labels[count]
        if (mode.physical == 0):
            physical_modes_in_order.append(mode)
            physical_modes_list.append(mode)
            physical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        else:
            mathematical_coordinates.append([mode.delta_frequency, mode.delta_damping, mode.delta_mac])
        count += 1
    physical_modes_dict[order] = physical_modes_in_order
    num_modes += len(physical_modes_in_order)

physical_coordinates = np.array(physical_coordinates)
mathematical_coordinates = np.array(mathematical_coordinates)
physical_modes_list = np.array(physical_modes_list)


In [14]:
print(physical_modes_list.shape[0])

1120


In [15]:
%matplotlib nbagg
#Making a 3D scatterplot of frequency, damping and MAC number divided into physical and mathematical clusters
fig = plt.figure()
ax = plt.axes(projection='3d')

fg = ax.scatter3D(physical_coordinates[:,0], physical_coordinates[:,1], physical_coordinates[:,2], c="blue", label= "Physical modes")
ax.scatter3D(mathematical_coordinates[:,0], mathematical_coordinates[:,1], mathematical_coordinates[:,2], c="red", label="Mathematical modes")
ax.legend()
ax.set_xlabel("Frequency")
ax.set_ylabel("Damping")
ax.set_zlabel("MAC")

print("Number of physical modes = " + str(physical_coordinates.shape[0]))
print("Number of mathematical modes = " +str(mathematical_coordinates.shape[0]))

<IPython.core.display.Javascript object>

Number of physical modes = 1120
Number of mathematical modes = 90


In [16]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(physical_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>

# Task 3
### Task 3.1 Detecting structural modes by hierachical clustering
Using hierarchical clustering with a signle linkage method with a cut-off-distance of $d_c=0.04$
$$ d_{c_{i,j}} = d \lambda_{i,j} + (1- MAC_{i,j}) $$

In [17]:
def distance_matrix(modes: np.ndarray) -> np.ndarray:
    """
    Computes the distance matrix between structural modes.
    The distance between two modes i and j is defined as:
    dc_ij = delta_eigenvalues_ij + (1 - MAC_ij)
    Parameters
    ----------
    modes: 1d numpy array with elements of class Mode

    Returns: 2d numpy array that is a distance matrix
    -------

    """
    #Preallocatig matrix to store the distances in
    dist_matrix = np.zeros((modes.shape[0], modes.shape[0]))

    for i in range(0, dist_matrix.shape[0]-1):
        for j in range(0, dist_matrix.shape[1]-1):
            #Computing the distance at each element
            if i != j:
                dist_matrix[i,j] = np.abs((modes[i].eigenvalue - modes[j].eigenvalue)/np.max([np.abs(modes[i].eigenvalue), np.abs(modes[j].eigenvalue)])) + (1 - strid.utils.modal_assurance_criterion(modes[i].v, modes[j].v))

    return dist_matrix

In [18]:
# https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html

# https://www.youtube.com/watch?v=v7oLMvcxgFY&ab_channel=KindsonTheTechPro

import strid
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


#Since we want to define our own distance between two poles, we must compute a distance matrix we can feed into the clustering algorithm.


In [19]:
distance = distance_matrix(physical_modes_list)

In [20]:
#Check the integrity of the distance matrix
print(np.max(distance))
print(np.mean(distance))
print(np.min(distance))

2.9971465262719463
1.8547502786001813
0.0


In [21]:
plt.figure()
plt.title("Dendogram")
dendogram = sch.dendrogram(sch.linkage(distance, method = 'ward'))

<IPython.core.display.Javascript object>

/var/folders/xg/ddkjb2x96w167hqnv_gjv_v40000gn/T/ipykernel_77412/2090417883.py:3: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  dendogram = sch.dendrogram(sch.linkage(distance, method = 'ward'))


In [22]:
#Perform the actual clustering
hc = AgglomerativeClustering(n_clusters=None, affinity='precomputed', linkage='complete', distance_threshold=0.04)
#The agglomerative clustering algorithm only finds 1 non-structural mode with single linkage method. Works better so far with 'complete' or 'average'.

In [23]:
y_hc = hc.fit_predict(distance)
print(type(y_hc))
print(np.max(y_hc))

<class 'numpy.ndarray'>
338


In [24]:
hierarchy = y_hc.tolist()
print(len(y_hc))
print(type(hierarchy))
print(hierarchy)

num_modes_in_hierarchy = np.zeros((np.max(y_hc) + 1, 3))
for i in range(0, len(num_modes_in_hierarchy)):
    num_modes_in_hierarchy[i, 0] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 1] = hierarchy.count(i)
    num_modes_in_hierarchy[i, 2] = i

print((num_modes_in_hierarchy))

1120
<class 'list'>
[334, 335, 49, 98, 309, 308, 175, 176, 130, 64, 333, 331, 28, 114, 191, 127, 104, 89, 75, 144, 145, 71, 184, 183, 57, 58, 24, 15, 67, 68, 69, 266, 285, 284, 338, 337, 160, 79, 300, 45, 59, 80, 52, 16, 13, 4, 56, 27, 177, 178, 49, 98, 87, 234, 236, 231, 229, 130, 64, 28, 114, 191, 127, 104, 89, 145, 71, 75, 144, 261, 245, 57, 58, 307, 306, 67, 68, 69, 266, 6, 2, 322, 246, 160, 79, 300, 45, 59, 80, 52, 16, 13, 4, 56, 27, 49, 98, 201, 199, 233, 232, 249, 9, 14, 153, 297, 281, 280, 191, 127, 104, 89, 75, 144, 88, 90, 57, 58, 24, 15, 67, 68, 69, 266, 6, 2, 273, 272, 160, 79, 300, 45, 59, 80, 56, 27, 52, 16, 13, 4, 149, 74, 29, 61, 187, 287, 271, 9, 14, 189, 239, 172, 115, 191, 127, 104, 89, 75, 144, 88, 90, 57, 58, 24, 15, 39, 10, 69, 266, 311, 310, 282, 279, 160, 79, 300, 45, 59, 80, 56, 27, 52, 16, 13, 4, 29, 61, 149, 74, 251, 257, 256, 318, 317, 211, 8, 3, 191, 127, 104, 89, 75, 144, 57, 58, 132, 65, 97, 48, 69, 266, 39, 10, 278, 258, 240, 238, 150, 60, 300, 45, 59, 8

In [25]:
structural_coordinates = []
non_structural_coordinates = []

for i in range(0, len(y_hc)):
    if y_hc[i] == 1:
        structural_coordinates.append(physical_coordinates[i,:])
    else:
        non_structural_coordinates.append(physical_coordinates[i,:])

structural_coordinates = np.array(structural_coordinates)
non_structural_coordinates = np.array(non_structural_coordinates)

In [26]:
#Assign hierarchy to each mode
count = 0
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    for mode in modes_of_order:
        mode.cluster = hierarchy[count]
        count += 1

# Task 4
### Task 4.1

In [27]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(num_modes_in_hierarchy[:,:2])
labels2 = kmeans.labels_

In [28]:
print(labels2)
print(len(labels2))

[0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
339


In [29]:
structural_hierarchies = []

for i in range(0, len(labels2)):
    if labels2[i] == 1:
        structural_hierarchies.append(i)

print(structural_hierarchies)

[4, 13, 17, 27, 36, 56, 57, 58, 69, 75, 89, 104, 127, 144, 191, 266]


In [30]:
#Assign hierarchy to each mode
structural_modes_dict = {}
for order in range(49, 5, -1):
    modes_of_order = physical_modes_dict[order]
    structural_modes_in_order = []
    for mode in modes_of_order:
        if mode.cluster in structural_hierarchies:
            structural_modes_in_order.append(mode)

    structural_modes_dict[order] = structural_modes_in_order

In [31]:
%matplotlib nbagg
#Making a new stabilization diagram with the physical modes
stabdiag = strid.StabilizationDiagram()
stabdiag.plot(structural_modes_dict)

f, psd = ssid.psdy(nperseg=2**10)

stabdiag.axes_psd.semilogy(f, np.trace(np.abs(psd)), color=(0., 0., 0., .5), lw=.3)

<IPython.core.display.Javascript object>